In [ ]:
#!jupyter nbconvert --to=python Chl_F1_F2_Comparison_python3.ipynb

In [ ]:
%matplotlib inline
%autosave 5

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
from netCDF4 import Dataset
from scipy.interpolate import griddata

#import pickle
#import scipy.io as spio


import pyfesom2 as pf

#import skill_metrics as sm
#from plot_Taylor_normalized import plt_Taylor_norm

import cartopy.crs as ccrs
import cartopy.feature as cfeature

def mygrid(m):
    '''
    Draw cartopy natural earth feature to axis m
    '''
    #m.coastlines(resolution='110m', color='black', linewidth=1)
    land_poly = cfeature.NaturalEarthFeature('physical', 'land', '110m',edgecolor='k',facecolor=cfeature.COLORS['land'])
    m.add_feature(land_poly)

In [ ]:
verbose = False

use_mg = True # build Chl:C ratio on [mg]:[mg]

# model version FESOM2 
id_label = 'mo12' # refers to run name, used for plotting

# period of analysis (when changing: load FESOM 1.4 data separetely!)
if(id_label == 'mo12'):
    first_year = 1970
    last_year  = 1980
else:
    first_year = 1960
    last_year  = 1970
    
#years = [first_year,last_year]
years = np.arange(first_year, last_year+1,1)

# figure settings
figyears = '{0} to {1}'.format(first_year,last_year)
print(figyears)

savepath = '/home/ollie/mozeisin/evaluation/mo_files/'


# define regular mesh
lat         = np.arange(-89.5,90.,1.)
lon         = np.arange(-179.5,180.,1.)
latdic, londic = np.meshgrid(lat, lon)

# load FESOM1.4 data

In [ ]:
dumppath = '/work/ollie/mozeisin/evaluation/temp_data/'

if(id_label == 'mo12'):
    print('Loading FESOM 1 data for 1970 to 1980')
    FESOM14_PhyChl_mg_m3 = np.load(dumppath + 'FESOM1.4_1970_1980_PhyChl_mg_m3.npy')
    FESOM14_PhyC_mmol_m3 = np.load(dumppath + 'FESOM1.4_1970_1980_PhyC_mmol_m3.npy')

    FESOM14_DiaChl_mg_m3 = np.load(dumppath + 'FESOM1.4_1970_1980_DiaChl_mg_m3.npy')
    FESOM14_DiaC_mmol_m3 = np.load(dumppath + 'FESOM1.4_1970_1980_DiaC_mmol_m3.npy')
    
else:
    print('Loading FESOM 1 data for 1960 to 1970')
    FESOM14_PhyChl_mg_m3 = np.load(dumppath + 'FESOM1.4_1960_1970_PhyChl_mg_m3.npy')
    FESOM14_PhyC_mmol_m3 = np.load(dumppath + 'FESOM1.4_1960_1970_PhyC_mmol_m3.npy')

    FESOM14_DiaChl_mg_m3 = np.load(dumppath + 'FESOM1.4_1960_1970_DiaChl_mg_m3.npy')
    FESOM14_DiaC_mmol_m3 = np.load(dumppath + 'FESOM1.4_1960_1970_DiaC_mmol_m3.npy')

In [ ]:
# convert FESOM 1.4 data -------------------------------------------------------------------------------------
if(use_mg == False):
    print('***\nFESOM 1.4 Chla data is converted to [mmol Chla/m3]...\n***')
    #Chl.a : 893.509 g/mol
    conv = 1/893.509
    ChlAfesom1 = FESOM14_PhyChl_mg_m3 * conv # mg/m3 to mmol/m3 
    DiaChlfesom1 = FESOM14_DiaChl_mg_m3 * conv
    
    PhyCfesom1 = FESOM14_PhyC_mmol_m3
    DiaCfesom1 = FESOM14_DiaC_mmol_m3
    
    unitfesomChl_f1 = 'FESOM 1.4 Chl.a [mmol m$^{-3}$]'
    unitfesomC_f1 = 'FESOM 1.4 intracellular C [mmol m$^{-3}$]'
    figlabel_f1 = 'molar Chla:C'
    
elif(use_mg == True):    
    print('***\nFESOM 1.4 Phyto carbon data is converted to [mg C/m3]...\n***')
    #C : 12.01 g/mol
    conv = 12.01
    PhyCfesom1 = FESOM14_PhyC_mmol_m3 * conv # mmol/m3 to mg/m3
    DiaCfesom1 = FESOM14_DiaC_mmol_m3 * conv
    
    ChlAfesom1 = FESOM14_PhyChl_mg_m3
    DiaChlfesom1 = FESOM14_DiaChl_mg_m3
    
    unitfesomC_f1 = 'FESOM 1.4 intracellular C [mg m$^{-3}$]'
    unitfesomChl_f1 = 'FESOM 1.4 Chl.a [mg m$^{-3}$]'
    figlabel_f1 = 'Chla : C [g g$^{-1}$]'
    
else: 
    print('***\nFESOM 1.4 Chla and C data are not converted, assuming [mg Chla/m3] and [mmol C m$^{-3}$]...\n***')
    unitfesomChl_f1 = 'FESOM Chl.a [mg m$^{-3}$]'
    unitfesomC_f1 = 'FESOM intracellular C [mmol m$^{-3}$]'
    


# load FESOM2 data

In [ ]:
# run specification -------------------------------------------------------------------------------------  
# already stated above: #id_label = 'mo12' # refers to run name, used for plotting

runid      =  'fesom'
layerwise    = False
resultpath = '/work/ollie/mozeisin/results/f2r1.2/' + id_label
meshpath = '/work/ollie/mozeisin/mesh/mesh_fesom2.0/core2_meanz'

print('processing {0} for years {1} to {2}'.format(resultpath,years[0],years[-1]))

# number of upper ocean layers for surface mean
n_levels = 1

# load FESOM mesh -------------------------------------------------------------------------------------
mesh       = pf.load_mesh(meshpath)

lon_fesom = mesh.x2
lat_fesom = mesh.y2        

# load FESOM Nanophyto C data -------------------------------------------------------------------------------------        
if verbose:
    ncFESOMPhyC = resultpath + '/PhyC.fesom.1948.nc'
    !ncdump -h $ncFESOMPhyC

PhyCfesom = pf.get_data(resultpath, "PhyC", years, mesh, 
                       how="mean", compute=True, runid=runid, silent=True)

PhyCfesom_surf_in = np.nanmean(PhyCfesom[:,:n_levels],axis=1)

# load FESOM Diatom C data -------------------------------------------------------------------------------------        
if verbose:
    ncFESOMDiaC = resultpath + '/DiaC.fesom.1948.nc'
    !ncdump -h $ncFESOMDiaC

DiaCfesom = pf.get_data(resultpath, "DiaC", years, mesh, 
                       how="mean", compute=True, runid=runid, silent=True)

DiaCfesom_surf_in = np.nanmean(DiaCfesom[:,:n_levels],axis=1)


# load FESOM Nanophyto Chl.a data -------------------------------------------------------------------------------------        
if verbose:
    ncFESOMChl = resultpath + '/ChlA.fesom.1948.nc'
    !ncdump -h $ncFESOMChl

ChlAfesom = pf.get_data(resultpath, "ChlA", years, mesh, 
                       how="mean", compute=True, runid=runid, silent=True)

ChlAfesom_surf_in = np.nanmean(ChlAfesom[:,:n_levels],axis=1)

# mean over whole water column
#ChlAfesom_mean = np.nanmean(ChlAfesom, axis=1)

labelfesomNano = 'FESOM Nanophyto Chl.a {0}-{1}'.format(years[0],years[-1])        
#unitfesomNano = 'Chl.a [mmol m$^{-3}$]' 

# load FESOM Diatom Chl.a data -------------------------------------------------------------------------------------
if verbose:
    ncFESOMDiaChl = resultpath + '/DiaChl.fesom.1948.nc'
    !ncdump -h $ncFESOMDiaChl

DiaChlfesom = pf.get_data(resultpath, "DiaChl", years, mesh, 
                       how="mean", compute=True, runid=runid, silent=True)

labelfesomDia = 'FESOM Diatom Chl.a {0}-{1}'.format(years[0],years[-1])

# surface: mean over n mesh levels
DiaChlfesom_surf_in = np.nanmean(DiaChlfesom[:,:n_levels],axis=1)

# Convert FESOM 2

In [ ]:
# convert FESOM2 CHl.a data -------------------------------------------------------------------------------------
# #########################
# FESOM2 outputs the Chl data in mg/m3 already, wrong units set in netcdf !?!?!
# #########################

if(use_mg == False):
    print('***\nFESOM Chla data is converted to [mmol Chla/m3]...\n***')
    # initial FESOM Chla assuming [mg Chla/m3]
    #Chl.a : 893.509 g/mol
    conv = 1/893.509
    ChlAfesom_surf = ChlAfesom_surf_in * conv # mg/m3 to mmol/m3 
    DiaChlfesom_surf = DiaChlfesom_surf_in * conv
    
    PhyCfesom_surf = PhyCfesom_surf_in # mmol/m3 
    DiaCfesom_surf = DiaCfesom_surf_in 
    
    unitfesomChl = 'FESOM Chl.a [mmol m$^{-3}$]'
    unitfesomC = 'FESOM intracellular C [mmol m$^{-3}$]'
    figlabel = 'molar C:Chla'
    
elif(use_mg == True):    
    print('***\nFESOM Phyto carbon data is converted to [mg C/m3]...\n***')
    # initial FESOM Chla assuming [mg Chla/m3]
    
    #C : 12.01 g/mol
    conv = 12.01
    PhyCfesom_surf = PhyCfesom_surf_in * conv # mmol/m3 to mg/m3
    DiaCfesom_surf = DiaCfesom_surf_in * conv
    
    ChlAfesom_surf = ChlAfesom_surf_in # mg/m3 
    DiaChlfesom_surf = DiaChlfesom_surf_in
    
    unitfesomC = 'FESOM intracellular C [mg m$^{-3}$]'
    unitfesomChl = 'FESOM Chl.a [mg m$^{-3}$]'
    figlabel = 'C:Chla [g g$^{-1}$]'
    
else: 
    print('***\nFESOM Chla and C data are not converted, assuming [mg Chla/m3] and [mmol C m$^{-3}$]...\n***')
    unitfesomChl = 'FESOM Chl.a [mg m$^{-3}$]'
    unitfesomC = 'FESOM intracellular C [mmol m$^{-3}$]'


print(figlabel)

# Control FESOM 2 ratio on original grid

In [ ]:
if verbose:  # time intensive!    
    fig_control_f2 = pf.plot(mesh, [PhyCfesom_surf/ChlAfesom_surf, DiaCfesom_surf/DiaChlfesom_surf],
        rowscol = (1,2),
        figsize = (20,20),
        levels = (0,1e3,101),
        mapproj = 'rob',
        units = figlabel + '\n' + id_label,
        titles = ['FESOM 2 Nanophyto C:Chl','FESOM 2 Diatom C:Chl'])

    if False:      
        savepath_aux = savepath+'Ratio_C_Chl_onlyFESOM2_origMesh_'+id_label+'_'+str(years[0])+'to'+str(years[-1])+'.png'
        print('saving to '+savepath_aux)

        plt.savefig(savepath_aux,dpi = 300, bbox_inches='tight')

# Interpolate FESOM 2 to 1x1

In [ ]:
# interpolate FESOM CHl.a to regular -------------------------------------------------------------------------------------
PhyCfesom_surf_interp = pf.fesom2regular(
        data = PhyCfesom_surf,
        mesh = mesh,
        lons = londic, 
        lats = latdic)

DiaCfesom_surf_interp = pf.fesom2regular(
        data = DiaCfesom_surf,
        mesh = mesh,
        lons = londic, 
        lats = latdic)

ChlAfesom_surf_interp = pf.fesom2regular(
        data = ChlAfesom_surf,
        mesh = mesh,
        lons = londic, 
        lats = latdic)

DiaChlfesom_surf_interp = pf.fesom2regular(
        data = DiaChlfesom_surf,
        mesh = mesh,
        lons = londic, 
        lats = latdic)

# Nanophyto + Diatoms: TOTAL CHLOROPHYLL -------------------------------------------------------------------------------------
ChlAfesom_surf_interp_log10 = np.log10(ChlAfesom_surf_interp)
#DiaChlfesom_surf_interp_log10 = np.log10(DiaChlfesom_surf_interp)

Chl_total = ChlAfesom_surf_interp + DiaChlfesom_surf_interp
#Chl_total_log10 = np.log10(Chl_total)

if verbose: # interpolation check
    
    # nanophyto C
    print('\nFESOM PhyC interpolation check:\noriginal min {0:5.4f} max {1:5.4f} mean {2:5.4f} \ninterpol min {3:5.4f} max {4:5.4f} mean {5:5.4f}'.format(
        np.nanmin(PhyCfesom_surf),np.nanmax(PhyCfesom_surf),np.nanmean(PhyCfesom_surf),
        np.nanmin(PhyCfesom_surf_interp),np.nanmax(PhyCfesom_surf_interp),np.nanmean(PhyCfesom_surf_interp)))
    
    # Diatom C
    print('\nFESOM DiaC interpolation check:\noriginal min {0:5.4f} max {1:5.4f} mean {2:5.4f} \ninterpol min {3:5.4f} max {4:5.4f} mean {5:5.4f}'.format(
        np.nanmin(DiaCfesom_surf),np.nanmax(DiaCfesom_surf),np.nanmean(DiaCfesom_surf),
        np.nanmin(DiaCfesom_surf_interp),np.nanmax(DiaCfesom_surf_interp),np.nanmean(DiaCfesom_surf_interp)))
    
    # Chlorophyll
    Chl_total_preinterp = ChlAfesom_surf + DiaChlfesom_surf

    print('\nFESOM Chl interpolation check:\noriginal min {0:5.4f} max {1:5.4f} mean {2:5.4f} \ninterpol min {3:5.4f} max {4:5.4f} mean {5:5.4f}'.format(
        np.nanmin(Chl_total_preinterp),np.nanmax(Chl_total_preinterp),np.nanmean(Chl_total_preinterp),
        np.nanmin(Chl_total),np.nanmax(Chl_total),np.nanmean(Chl_total)))

    fig = plt.figure(figsize=(10,10))
    ax1 = plt.subplot(projection = ccrs.Robinson())

    m1 = plt.pcolormesh(londic, latdic, Chl_total, 
        transform = ccrs.PlateCarree(),
        norm=colors.LogNorm(vmin=np.nanmin(Chl_total), 
                            vmax=np.nanmax(Chl_total)),
        cmap='viridis')

    ax1.coastlines(resolution='110m', color='black', linewidth=1)

    cbar = fig.colorbar(m1,orientation = 'horizontal',fraction=0.1, pad=0.1) 
    cbar.set_label('Interpolated '+unitfesomChl, fontsize=20)

# FESOM 1 & FESOM 2 Chlorophyll

In [ ]:
# plotting

levels = np.array([0,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,
                           0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,
                           1,2,3,4,5,7])
ticks = [0,0.01,0.03,0.05,0.07,0.1,0.3,0.5,0.7,1,3,5,7]
ticks_label = ['0','0.01','0.03','0.05','0.07','0.1','0.3','0.5','0.7','1','3','5','7']    

fig, axes = plt.subplots(2,2, 
                         subplot_kw=dict(projection=ccrs.Robinson()),
                         gridspec_kw={'hspace': 0.01, 'wspace': 0.1},
                         figsize=(18,13))
# FESOM 1 Nanophyto
m = axes[0,0]
f = m.pcolormesh(londic, latdic, ChlAfesom1, 
                   transform = ccrs.PlateCarree(),
                   norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))

m.set_title('FESOM 1 Nanophytoplankton', fontsize=16)
mygrid(m)

# FESOM 1 Diatom
m = axes[1,0]
f = m.pcolormesh(londic, latdic, DiaChlfesom1, 
                   transform = ccrs.PlateCarree(),
                   norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))

m.set_title('FESOM 1 Diatom', fontsize=16)
mygrid(m)

# FESOM 2 Nanophyto
m = axes[0,1]
f = m.pcolormesh(londic, latdic, ChlAfesom_surf_interp, 
                   transform = ccrs.PlateCarree(),
                   norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))

m.set_title('FESOM 2 Nanophytoplankton', fontsize=16)
mygrid(m)

# FESOM 2 Diatom
m = axes[1,1]
f = m.pcolormesh(londic, latdic, DiaChlfesom_surf_interp, 
                   transform = ccrs.PlateCarree(),
                   norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))

m.set_title('FESOM 2 Diatom', fontsize=16)
mygrid(m)

#########################################
# add one colorbar for first row plots below figure
cbar = fig.colorbar(f,
                    ax = axes, 
                    #location ='bottom',
                    orientation = 'horizontal',
                    ticks = ticks,
                    fraction=0.05, pad=0.05) 
#cbar.ax.tick_params(labelsize=14)
cbar.ax.set_xticklabels(ticks_label, fontsize=16) 
cbar.set_label('Chl [mg m$^{-3}$]\n'+figyears+'\nFESOM 2: '+id_label, fontsize=16)

if True:                
    save_aux = savepath+'Chl_FESOM1_FESOM2_'+id_label+'_'+str(years[0])+'to'+str(years[-1])+'.png'
    print('Saving to '+save_aux)
    plt.savefig(save_aux, 
            dpi = 300, bbox_inches='tight')

# FESOM 2 Chl:C

In [ ]:
# Nanophytoplankton
print('use_mg = {0}'.format(use_mg))
phy_subtext = ('\nNanophyto global mean {0:4.2f}, min {1:4.2f}, max {2:4.2f}'.format(
    np.nanmean(PhyCfesom_surf_interp/ChlAfesom_surf_interp),np.nanmin(PhyCfesom_surf_interp/ChlAfesom_surf_interp),np.nanmax(PhyCfesom_surf_interp/ChlAfesom_surf_interp))
     )
print(phy_subtext)

In [ ]:
# Diatom
print('use_mg = {0}'.format(use_mg))
dia_subtext = ('\nDiatom global mean {0:4.2f}, min {1:4.2f}, max {2:4.2f}'.format(
    np.nanmean(DiaCfesom_surf_interp/DiaChlfesom_surf_interp),np.nanmin(DiaCfesom_surf_interp/DiaChlfesom_surf_interp),np.nanmax(DiaCfesom_surf_interp/DiaChlfesom_surf_interp))
     )
print(dia_subtext)

In [ ]:
## C:Chl
vmin,vmax = 0, 1e3 # for C:Chl
# use norm in pcolormesh to plot data on log_transformed scale!

fig, axes = plt.subplots(1,2, 
                         subplot_kw=dict(projection=ccrs.Robinson()),
                         gridspec_kw={'hspace': 0.01, 'wspace': 0.1},
                         figsize=(16,12))

# FESOM 2 Nanophyto
m3 = axes[0]
f3 = m3.pcolormesh(londic, latdic, PhyCfesom_surf_interp/ChlAfesom_surf_interp, 
                   transform = ccrs.PlateCarree(),
                   #norm=colors.LogNorm(vmin=vmin,vmax=vmax),
                   vmin=vmin,vmax=vmax,
                   cmap = 'Spectral_r')
mygrid(m3)
m3.set_title('FESOM 2 Nanophyto', fontsize=16)

# FESOM 1 Diatom
m4 = axes[1]
f4 = m4.pcolormesh(londic, latdic, DiaCfesom_surf_interp/DiaChlfesom_surf_interp, 
                   transform = ccrs.PlateCarree(),
                   #norm=colors.LogNorm(vmin=vmin,vmax=vmax),
                   vmin=vmin,vmax=vmax,
                   cmap = 'Spectral_r')
mygrid(m4)
m4.set_title('FESOM 2 Diatom', fontsize=16)

# add one colorbar below figure
cbar = fig.colorbar(f3,
                    ax = axes[:2], 
                    location ='bottom',
                    #ticks = ticks,
                    fraction=0.05, pad=0.05) 
cbar.ax.tick_params(labelsize=14)
#cbar.ax.set_xticklabels(ticks_label, fontsize=16) 
cbar.set_label('C:Chl [g g$^{-1}$]\n'+figyears+'\n'+id_label+phy_subtext+dia_subtext, fontsize=16)

if True:                
    savepath_aux = savepath+'Ratio_C_Chl_onlyFESOM2_'+id_label+'_'+str(years[0])+'to'+str(years[-1])+'.png'
    print('saving to '+savepath_aux)
    
    plt.savefig(savepath_aux,dpi = 300, bbox_inches='tight')

In [ ]:
stop

# !!! Proceeed only with control run !!!

# FESOM 1 & 2 C:Chl

In [ ]:
## C:Chl
vmin,vmax = 0, 1e3 # for C:Chl


fig, axes = plt.subplots(2,2, 
                         subplot_kw=dict(projection=ccrs.Robinson()),
                         gridspec_kw={'hspace': 0.01, 'wspace': 0.1},
                         figsize=(18,12))
# FESOM 1 Nanophyto
m1 = axes[0,0]
f1 = m1.pcolormesh(londic, latdic, PhyCfesom1/ChlAfesom1, 
                   transform = ccrs.PlateCarree(),
                   #norm=colors.LogNorm(vmin=vmin,vmax=vmax),
                   vmin=vmin,vmax=vmax,
                   cmap = 'Spectral_r')
mygrid(m1)
m1.set_title('FESOM 1 Nanophyto', fontsize=16)

# FESOM 1 Diatom
m2 = axes[1,0]
f2 = m2.pcolormesh(londic, latdic, DiaCfesom1/DiaChlfesom1, 
                   transform = ccrs.PlateCarree(),
                   #norm=colors.LogNorm(vmin=vmin,vmax=vmax),
                   vmin=vmin,vmax=vmax,
                   cmap = 'Spectral_r')
mygrid(m2)
m2.set_title('FESOM 1 Diatom', fontsize=16)

# FESOM 2 Nanophyto
m3 = axes[0,1]
f3 = m3.pcolormesh(londic, latdic, PhyCfesom_surf_interp/ChlAfesom_surf_interp, 
                   transform = ccrs.PlateCarree(),
                   #norm=colors.LogNorm(vmin=vmin,vmax=vmax),
                   vmin=vmin,vmax=vmax,
                   cmap = 'Spectral_r')
mygrid(m3)
m3.set_title('FESOM 2 Nanophyto', fontsize=16)

# FESOM 1 Diatom
m4 = axes[1,1]
f4 = m4.pcolormesh(londic, latdic, DiaCfesom_surf_interp/DiaChlfesom_surf_interp, 
                   transform = ccrs.PlateCarree(),
                   #norm=colors.LogNorm(vmin=vmin,vmax=vmax),
                   vmin=vmin,vmax=vmax,
                   cmap = 'Spectral_r')
mygrid(m4)
m4.set_title('FESOM 2 Diatom', fontsize=16)

# add one colorbar below figure
cbar = fig.colorbar(f1,
                    ax = axes[1,:2], 
                    location ='bottom',
                    #ticks = ticks,
                    fraction=0.1, pad=0.1) 
cbar.ax.tick_params(labelsize=14)
#cbar.ax.set_xticklabels(ticks_label, fontsize=16) 
cbar.set_label('C:Chl [g g$^{-1}$]\n'+figyears+'\nFESOM 2: '+id_label, fontsize=16)

if True:                
    save_aux = savepath+'Ratio_C_Chl_FESOM1_FESOM2_'+str(years[0])+'to'+str(years[-1])+'.png'
    print('Saving to '+save_aux)
    
    plt.savefig(save_aux, dpi = 300, bbox_inches='tight')

# Chl:C

In [ ]:
### Chl:C
vmin, vmax = 0, 0.055 # for Chl:C


fig, axes = plt.subplots(2,2, 
                         subplot_kw=dict(projection=ccrs.Robinson()),
                         gridspec_kw={'hspace': 0.01, 'wspace': 0.1},
                         figsize=(18,12))
# FESOM 1 Nanophyto
m1 = axes[0,0]
f1 = m1.pcolormesh(londic, latdic, ChlAfesom1/PhyCfesom1, 
                   transform = ccrs.PlateCarree(),
                   #norm=colors.LogNorm(vmin=vmin,vmax=vmax),
                   vmin=vmin,vmax=vmax,
                   cmap = 'viridis')
mygrid(m1)
m1.set_title('FESOM 1 Nanophyto', fontsize=16)

# FESOM 1 Diatom
m2 = axes[1,0]
f2 = m2.pcolormesh(londic, latdic, DiaChlfesom1/DiaCfesom1, 
                   transform = ccrs.PlateCarree(),
                   #norm=colors.LogNorm(vmin=vmin,vmax=vmax),
                   vmin=vmin,vmax=vmax,
                   cmap = 'viridis')
mygrid(m2)
m2.set_title('FESOM 1 Diatom', fontsize=16)

# FESOM 2 Nanophyto
m3 = axes[0,1]
f3 = m3.pcolormesh(londic, latdic, ChlAfesom_surf_interp/PhyCfesom_surf_interp, 
                   transform = ccrs.PlateCarree(),
                   #norm=colors.LogNorm(vmin=vmin,vmax=vmax),
                   vmin=vmin,vmax=vmax,
                   cmap = 'viridis')
mygrid(m3)
m3.set_title('FESOM 2 Nanophyto', fontsize=16)

# FESOM 2 Diatom
m4 = axes[1,1]
f4 = m4.pcolormesh(londic, latdic, DiaChlfesom_surf_interp/DiaCfesom_surf_interp, 
                   transform = ccrs.PlateCarree(),
                   #norm=colors.LogNorm(vmin=vmin,vmax=vmax),
                   vmin=vmin,vmax=vmax,
                   cmap = 'viridis')
mygrid(m4)
m4.set_title('FESOM 2 Diatom', fontsize=16)

# add one colorbar below figure
cbar = fig.colorbar(f1,
                    ax = axes[1,:2], 
                    location ='bottom',
                    #ticks = ticks,
                    fraction=0.1, pad=0.1) 
cbar.ax.tick_params(labelsize=14)
#cbar.ax.set_xticklabels(ticks_label, fontsize=16) 
cbar.set_label('Chl:C [g g$^{-1}$]\n'+figyears+'\nFESOM 2: '+id_label, fontsize=16)

if True:                
    plt.savefig(savepath+'Ratio_Chl_C_FESOM1_FESOM2_'+str(years[0])+'to'+str(years[-1])+'.png', 
            dpi = 300, bbox_inches='tight')